In [1]:
import numpy as np
import cv2

In [2]:
def rgb_to_hue(r, g, b):
    [[[h, _, _]]] = cv2.cvtColor(np.array([[[b, g, r]]]).astype('uint8'), cv2.COLOR_BGR2HSV)
    return h

In [3]:
def get_hue(image):
    hue_sum = 0
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            hue_sum += image[i, j, 0]
    return hue_sum//(image.shape[0] * image.shape[1])

In [4]:
hues = [get_hue(cv2.imread("col/orange.png")), 
        get_hue(cv2.imread("col/yellow.png")),
        get_hue(cv2.imread("col/green.png")),
        get_hue(cv2.imread("col/white.png")), 
        get_hue(cv2.imread("col/blue.png")),
        get_hue(cv2.imread("col/red.png"))]

print(hues)
rngs = [(hues[i-1]+hues[i])//2 for i in range(1, len(hues))]
print(rngs)

[8, 26, 63, 99, 110, 176]
[17, 44, 81, 104, 143]


In [5]:
def detect_colors (src, tgt, hue1, hue2, col):
    lower = np.array([hue1, 50, 50])
    upper = np.array([hue2, 255, 255])
        
    mask = cv2.inRange(hsv, lower, upper)
    ret, thresh = cv2.threshold(mask, 127, 255, 0)
    _, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        cnt_area = cv2.contourArea(cnt)
        rect = cv2.minAreaRect(cnt)
        ((x, y),(h, w),_) = rect
        org = int(x)-15, int(y)
        if abs(w-h) < 20:
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            box_area = cv2.contourArea(box)  
            if box_area > 2560 and box_area < 256000 and 2*cnt_area > box_area:
                cv2.putText(tgt, col, org, cv2.FONT_HERSHEY_SIMPLEX, .5, (255, 255, 255))
                cv2.drawContours(tgt, [box], 0, (0,0,255), 2)

In [6]:
def detect_white (src, tgt):
    ret, thresh = cv2.threshold(src, 170, 255, 0)
    _, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        cnt_area = cv2.contourArea(cnt)
        rect = cv2.minAreaRect(cnt)
        ((x, y),(h, w),_) = rect
        org = int(x)-15, int(y)
        if abs(w-h) < 20:
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            box_area = cv2.contourArea(box)  
            if box_area > 2560 and box_area < 256000 and 2*cnt_area > box_area:
                cv2.putText(tgt, 'White', org, cv2.FONT_HERSHEY_SIMPLEX, .5, (255, 255, 255))
                cv2.drawContours(tgt, [box], 0, (0,0,255), 2)

In [7]:
def enhance(src):
    lab= cv2.cvtColor(src, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final

In [8]:
def power_law_trans(src, gamma):
    ret = src.copy()
    ret = (ret/255)**gamma
    ret = ret*255
    return ret

In [ ]:
cap = cv2.VideoCapture(-1)
while True:
    _, frame = cap.read()
    blur = cv2.medianBlur(frame, 1)
    #print(rgb_to_hue(frame[0, 0, 2], frame[0, 0, 1], frame[0, 0, 0]))
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
    gray = power_law_trans(gray, 1.5).astype('uint8');
    
    detect_colors(hsv, frame, 0, 17, 'Orange')
    detect_colors(hsv, frame, 18 , 44, 'Yellow')
    detect_colors(hsv, frame, 45, 81, 'Green')
    detect_colors(hsv, frame, 95, 120, 'Blue')
    detect_colors(hsv, frame, 144, 180, 'Red')
    detect_white(gray, frame)
    
    cv2.imshow('frame',frame)
    cv2.imshow('blur',blur)
    cv2.imshow('gray',gray)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cv2.destroyAllWindows()